In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive

In [ ]:
#connect ke gdrive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#load dataset
data1 = r'/content/drive/My Drive/TA/Kodingan/dataset/destinasi.xlsx'
data2 = r'/content/drive/My Drive/TA/Kodingan/dataset/rating_user.xlsx'

**Input User ID**

In [ ]:
# input id pengguna
print('Masukkan id pengguna >>>')
input_a = input()
input_a = int(input_a)



# cek data type
print(type(input_a))


Masukkan id pengguna >>>
34
<class 'int'>


**Content-based Filtering**

In [ ]:
#data1
data_cb = pd.read_excel(data1)
data_cb

,id_wisata,nama_wisata,kategori_wisata,alamat
0,0,Masjid Islamic Center,Situs Suci & Agama,"Jl. Langko no. 5, Mataram, Lombok Indonesia"
1,1,Lombok Epicentrum,Tempat Belanja,"Jl. Sriwijaya no. 333, Mataram, Lombok"
2,2,Hutan Sesaot,"Area Wisata Alam & Taman Margasatwa, Hutan","Batu Mekar, Lingsar, Kabupaten Lombok Barat, L..."
3,3,Pura Agung Narmada,Taman,"Lembuak, Narmada, Kabupaten Lombok Barat, Lombok"
4,4,Pura Lingsar,Tempat Menarik & Tengara,"Jl. Raya Lingsar, Batu Kumbung, Lingsar, Kabup..."
...,...,...,...,...
73,73,Pantai Gondang,Pantai,"Genggelang, Gangga, Kabupaten Lombok Utara, Lo..."
74,74,Pantai Ketapang,Pantai,"Jl. Raya Bayan, Selengen, Kayangan, Kabupaten ..."
75,75,Pantai Are Guling,Pantai,"Tumpak, Pujut, Kabupaten Lombok Tengah, Lombok"
76,76,Pantai Bangko-Bangko,Pantai,"Jalan Putih, Batu Putih, Sekotong Tengah, Kabu..."


In [ ]:
#preprocessing data
#1. check missing value
data_cb.isna().sum()

id_wisata          0
nama_wisata        0
kategori_wisata    0
alamat             0
dtype: int64

In [ ]:
#preprocessing data
import string 
import re 

#2. feature merge
data_cb['metadata'] = data_cb['kategori_wisata'].str.cat(data_cb['alamat'], sep=', ').to_frame()

#3. lower case
def lower_case(text):
  return text.casefold()

data_cb['metadata'] = data_cb['metadata'].apply(lower_case)

#4. remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

data_cb['metadata'] = data_cb['metadata'].apply(remove_number)

#5. remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

data_cb['metadata'] = data_cb['metadata'].apply(remove_punctuation)

data_cb.head()

,id_wisata,nama_wisata,kategori_wisata,alamat,metadata
0,0,Masjid Islamic Center,Situs Suci & Agama,"Jl. Langko no. 5, Mataram, Lombok Indonesia",situs suci agama jl langko no mataram lombok...
1,1,Lombok Epicentrum,Tempat Belanja,"Jl. Sriwijaya no. 333, Mataram, Lombok",tempat belanja jl sriwijaya no mataram lombok
2,2,Hutan Sesaot,"Area Wisata Alam & Taman Margasatwa, Hutan","Batu Mekar, Lingsar, Kabupaten Lombok Barat, L...",area wisata alam taman margasatwa hutan batu ...
3,3,Pura Agung Narmada,Taman,"Lembuak, Narmada, Kabupaten Lombok Barat, Lombok",taman lembuak narmada kabupaten lombok barat l...
4,4,Pura Lingsar,Tempat Menarik & Tengara,"Jl. Raya Lingsar, Batu Kumbung, Lingsar, Kabup...",tempat menarik tengara jl raya lingsar batu k...


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_distances

!pip install nltk
import nltk
nltk.download('punkt')

class rec_cb:
  def __init__(self, data, content_col):
    self.df = data_cb
    self.content_col = content_col
    self.encoder = None
    self.bank = None

  def fit(self):
    self.encoder = TfidfVectorizer(stop_words='english', tokenizer=word_tokenize)
    self.bank = self.encoder.fit_transform(self.df[self.content_col])
  
  def recommend(self, idx, topk=10):
    content = self.df.loc[idx, self.content_col]
    code = self.encoder.transform([content])
    dist = cosine_distances(code, self.bank)
    rec_idx = dist.argsort() [0, 0:(topk+1)]
    recom_cb = self.df.loc[rec_idx].nama_wisata
    df_cosine = pd.DataFrame({'nama_wisata': recom_cb, 'idx' : rec_idx})
    df_cosine['cosine_distance'] = dist[0,df_cosine.idx]
    self.df['cosine_distance'] = df_cosine['cosine_distance']
    return self.df.loc[self.df.cosine_distance<= 0.6]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
recomendasi_cb = rec_cb('data_cb', content_col = 'metadata')
recomendasi_cb.fit()

In [ ]:
hasil_cb = recomendasi_cb.recommend(input_b)
hasil_cb

,id_wisata,nama_wisata,kategori_wisata,alamat,metadata,cosine_distance
0,0,Masjid Islamic Center,Situs Suci & Agama,"Jl. Langko no. 5, Mataram, Lombok Indonesia",situs suci agama jl langko no mataram lombok...,0.000000
11,11,Pura Batu Bolong,Situs Suci & Agama,"Jl. Raya Senggigi, Senggigi, Lombok",situs suci agama jl raya senggigi senggigi lo...,0.548958
22,22,Desa Senaru,"Tempat Menarik & Tengara, Situs Suci & Agama","Jl. Pariwisata, Bayan, Kabupaten Lombok Utara,...",tempat menarik tengara situs suci agama jl p...,0.570720


In [ ]:
rec = hasil_cb.nama_wisata
rec

0     Masjid Islamic Center
11         Pura Batu Bolong
22              Desa Senaru
Name: nama_wisata, dtype: object

In [ ]:
df_cb_pred = pd.read_excel(data2)
df_cb_pred

,id_pengguna,nama_pengguna,id_wisata,nama_wisata,rating
0,0,Supardi,0,Masjid Islamic Center,5
1,1,DEUTZ18,0,Masjid Islamic Center,5
2,2,Kanz The Traveller,0,Masjid Islamic Center,5
3,3,tholibkhair,0,Masjid Islamic Center,5
4,4,Kiki Wiryono,0,Masjid Islamic Center,4
...,...,...,...,...,...
5689,253,rizal993,74,Pantai Ketapang,5
5690,289,tjutjukbs,74,Pantai Ketapang,4
5691,3637,Rita O,74,Pantai Ketapang,4
5692,3638,Shelin,75,Pantai Are Guling,5


In [ ]:
df_pengguna = df_cb_pred[(df_cb_pred['id_pengguna'] == 0)]
df_pengguna

,id_pengguna,nama_pengguna,id_wisata,nama_wisata,rating
0,0,Supardi,0,Masjid Islamic Center,5


In [ ]:
df_cf_baru2 = pd.DataFrame({
    'id_pengguna' : df_pengguna["id_pengguna"],
    'nama_wisata' : df_pengguna["nama_wisata"],
    'rating' : df_pengguna["rating"]
})
df_cf_baru2

,id_pengguna,nama_wisata,rating
0,0,Masjid Islamic Center,5


In [ ]:
id_pengguna = input_a

In [ ]:
!pip install scikit-surprise
from surprise import SVD, Dataset, Reader
from surprise.model_selection import cross_validate

In [ ]:
data_pred = Dataset.load_from_df(df_cf_baru2, Reader())
trainset2 = data_pred.build_full_trainset()

In [ ]:
model2 = SVD()
model2.fit(trainset2)

In [ ]:
rating_prediction_cb = [model2.predict(id_pengguna, nama_wisata).est for nama_wisata in rec]
df_prediksi = pd.DataFrame()
df_prediksi['nama_wisata'] = hasil_cb.nama_wisata
df_prediksi['cosine_distance'] = hasil_cb['cosine_distance']
df_prediksi['prediksi_rating_cb'] = rating_prediction_cb

**Collaborative Filtering**

In [ ]:
#data2
data2 = pd.read_excel(data2)
data2.head()

,id_pengguna,nama_pengguna,id_wisata,nama_wisata,rating
0,0,Supardi,0,Masjid Islamic Center,5
1,1,DEUTZ18,0,Masjid Islamic Center,5
2,2,Kanz The Traveller,0,Masjid Islamic Center,5
3,3,tholibkhair,0,Masjid Islamic Center,5
4,4,Kiki Wiryono,0,Masjid Islamic Center,4


In [ ]:
#Preprocessing

#1. Pilih Fitur yang dibutuhkan
df_cf = pd.DataFrame({
    'id_pengguna' : data2["id_pengguna"],
    'nama_wisata' : data2["nama_wisata"],
    'rating' : data2["rating"]
})
df_cf.head()

,id_pengguna,nama_wisata,rating
0,0,Masjid Islamic Center,5
1,1,Masjid Islamic Center,5
2,2,Masjid Islamic Center,5
3,3,Masjid Islamic Center,5
4,4,Masjid Islamic Center,4


In [ ]:
#Preprocessing

#2. check missing value
df_cf.isna().sum()

id_pengguna    0
nama_wisata    0
rating         0
dtype: int64

In [ ]:
!pip install scikit-surprise
from surprise import SVD, Dataset, Reader
from surprise.model_selection import cross_validate

class rec_cf:
  def __init__(self, data):
    self.df = df_cf
    self.all_destinations = self.df.nama_wisata.unique()
    self.model = None
  
  def fit(self):
    data = Dataset.load_from_df(df_cf, Reader())
    trainset = data.build_full_trainset()
    self.model = SVD()
    self.model.fit(trainset)

  def recommend(self, id_pengguna, topk = 77):
    rated = self.df[self.df.id_pengguna == id_pengguna].nama_wisata
    not_rated = [nama_wisata for nama_wisata in self.all_destinations if nama_wisata not in rated]
    score = [self.model.predict(id_pengguna, nama_wisata).est for nama_wisata in not_rated]
    result = pd.DataFrame({'nama_wisata': not_rated, 'prediksi_rating_cf' : score})
    return result.head(topk)

In [ ]:
rekomendasi_cf = rec_cf('data_cf')
rekomendasi_cf.fit()

In [ ]:
df_prediksi2 = rekomendasi_cf.recommend(id_pengguna=input_a)

**Hybrid Method**

In [ ]:
#preprocessing

#feature merge
data_hybrid = pd.merge(df_prediksi.sort_values(['nama_wisata'], ascending=[True]), df_prediksi2.sort_values(['nama_wisata'], ascending=[True]))
data_hybrid

,nama_wisata,cosine_distance,prediksi_rating_cb,prediksi_rating_cf
0,Desa Senaru,0.570720,5,4.471512
1,Masjid Islamic Center,0.000000,5,4.752971
2,Pura Batu Bolong,0.548958,5,4.160988


In [ ]:
def recommend(data):
  data['prediksi_rating_hybrid'] = (0.5 * data.prediksi_rating_cf) + (0.5 * data.prediksi_rating_cb)
  result = data.sort_values(['prediksi_rating_hybrid'], ascending=[False])
  return result

**Hasil Rekomendasi**

In [ ]:
#rekomendasi content-based filtering
df_prediksi

,nama_wisata,cosine_distance,prediksi_rating_cb
0,Masjid Islamic Center,0.000000,5
11,Pura Batu Bolong,0.548958,5
22,Desa Senaru,0.570720,5


In [ ]:
#rekomendasi collaborative filtering
df_prediksi2

,nama_wisata,prediksi_rating_cf
0,Masjid Islamic Center,4.752971
1,Lombok Epicentrum,4.352610
2,Hutan Sesaot,4.593304
3,Pura Agung Narmada,3.632915
4,Pura Lingsar,3.764422
...,...,...
72,Pantai Induk,4.435672
73,Pantai Gondang,4.457341
74,Pantai Ketapang,4.293717
75,Pantai Are Guling,4.418471


In [ ]:
#rekomendasi hybrid (teknik pembobotan)
recommend(data_hybrid)

,nama_wisata,cosine_distance,prediksi_rating_cb,prediksi_rating_cf,prediksi_rating_hybrid
1,Masjid Islamic Center,0.000000,5,4.752971,4.876486
0,Desa Senaru,0.570720,5,4.471512,4.735756
2,Pura Batu Bolong,0.548958,5,4.160988,4.580494
